In [ ]:
# !pip install "sap-llm-commons[all]"

In [ ]:
# !pip install ai-api-client-sdk

In [1]:
# proxy configuration
# from ipywidgets import widgets
import json
import os
import requests
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

C:\Users\I541988\AppData\Local\Temp\ipykernel_6780\161654179.py:6: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  import llm_commons.proxy.base


In [2]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [ ]:
# resource_group = widgets.Text(
#     value='default', # resource group
#     placeholder='Resource group of deployments',
#     description='',
#     disabled=False
# )

# resource_group

In [3]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [4]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ['ENDPOINT_URL'] = sk['endpoint']

llm_commons.proxy.resource_group = 'default'
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']
llm_commons.proxy.endpoint_url = os.environ['ENDPOINT_URL']

NameError: name 'resource_group' is not defined

In [ ]:
aic_proxy_client = AICoreProxyClient()


In [ ]:
response = requests.post(
        f'{os.environ["AICORE_LLM_AUTH_URL"]}/oauth/token',
        data={"grant_type": "client_credentials"},
        auth=(os.environ['AICORE_LLM_CLIENT_ID'], os.environ['AICORE_LLM_CLIENT_SECRET']),
        timeout=8000,
)
auth_token = response.json()["access_token"]
print (auth_token)

Deploy customized fine-tuned models

In [ ]:
import autogen

llm_config={
    "config_list":[
        {
            # "base_url":"is the same with restful api url api_key is the restful api auth token"
            "base_url":llm_commons.proxy.endpoint_url,
            "api_key": auth_token,
            "api_version":"2023-05-15",
            "api_type":"azure"
        }
    ],
    "temperature": 0,
    "model":"gpt-4"
}

In [ ]:
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
user_proxy= autogen.UserProxyAgent(
    name="user_proxy",
    max_consecutive_auto_reply=10,
    human_input_mode="NEVER",
    llm_config=llm_config,
       code_execution_config={
        "use_docker": False,
    },
)

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="""what is the whether in shanghai?""",
)